### Build Real World Agent

1. Detailed system prompts for better agent behavior
2. Create tools that integrate with external data
3. Model configuration for consistent responses
4. Structured output for predictable results
5. Conversational memory for chat-like interactions
6. Create and run the agent create a fully functional agent

In [2]:
from dotenv import load_dotenv
import os
from langchain.agents import create_agent

C:\Users\Ankit\AppData\Roaming\Python\Python314\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
# Load .env variables
load_dotenv()

True

In [4]:
# Read API key
gpt_api_key = os.getenv("OPENAI_API_KEY")

# Set the environment variable that OpenAI expects
if gpt_api_key:
    print("API Loaded Succesfully!")
else:
    raise ValueError("GPT_API not found in .env file")

API Loaded Succesfully!


### 1. Define the system prompt

##### The system prompt defines your agent’s role and behavior. Keep it specific and actionable:

In [1]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

### 2. Create tools

##### Tools let a model interact with external systems by calling functions you define. Tools can depend on runtime context and also interact with agent memory.
##### Notice below how the get_user_location tool uses runtime context:

In [5]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

@tool is a decorator that transforms a function into a tool, making it discoverable and callable within a Langchain environment. It allows you to define a function (like get_weather_for_location or get_user_location) and easily integrate it into a Langchain workflow.

@dataclass is a decorator that automatically generates methods like __init__, __repr__, etc., for a class. In this case, it simplifies the creation of the Context class, which is used to provide runtime context to tools.

### Configure your model

##### Set up your language model with the right parameters for your use case:

In [6]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=1000,
    timeout=30
    # ... (other params)
)

### Define response format

##### Optionally, define a structured response format if you need the agent responses to match a specific schema.

In [8]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

### Add memory

##### Add memory to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

### Create and run the agent

##### Now assemble your agent with all the components and run it!

In [12]:
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like Florida is putting on its best show with sunny weather! It's a bright day to soak up some rays!", weather_conditions='sunny')
ResponseFormat(punny_response="You're welcome! I'm always here to brighten your day with some weather puns!", weather_conditions=None)
